In [ ]:
import math
import xfacpy as xfac


def price_trinomial_call_ttcross(
    N,
    S0,
    K,
    r,
    T,
    u, m, d,
    pu, pm, pd,
    bond_dim=50,
    reltol=1e-8,
    max_sweeps=20,
    verbose=False,
):
    """
    Pricing di una call europea via trinomial tree + TT-cross (xfac).

    f(path) = E[discounted payoff] discreto, cioè
    f(indices) = e^{-rT} * Prob(path) * (S_T(path) - K)^+
    e poi facciamo direttamente sum1() sul TT.
    """

    if abs((pu + pm + pd) - 1.0) > 1e-10:
        raise ValueError("Le probabilità pu, pm, pd devono sommare a 1.")

    # Controllo minimo: il path tutto-up deve essere in-the-money,
    # altrimenti pivot1 non è un pivot "buono"
    ST_all_up = S0 * (u ** N)
    if ST_all_up <= K:
        raise ValueError(
            f"Il path tutto-up è ancora out-of-the-money (S_T={ST_all_up} <= K={K}). "
            "Scegli parametri diversi per u/S0/K oppure un pivot diverso."
        )

    disc = math.exp(-r * T)
    localDim = [3] * N

    def f(indices):
        S = S0
        prob = 1.0
        for move in indices:
            if move == 0:
                S *= d
                prob *= pd
            elif move == 1:
                S *= m
                prob *= pm
            else:  # move == 2
                S *= u
                prob *= pu
        payoff = max(S - K, 0.0)
        return disc * prob * payoff

    # Parametri TT-cross
    param = xfac.TensorCI2Param()
    param.bondDim = bond_dim
    param.reltol  = reltol

    # Scegliamo pivot1 = path tutto-up: [2,2,...,2]
    pivot = [2] * N
    p1 = param.pivot1
    # Cerco di capire se è una lista di interi o lista di liste
    if isinstance(p1, list) and len(p1) > 0 and isinstance(p1[0], list):
        # caso MultiIndex-like (lista di multiindici)
        param.pivot1 = [pivot]
    else:
        # caso semplice: un solo multiindice
        param.pivot1 = pivot

    # Costruzione CI2
    ci = xfac.TensorCI2(f, localDim, param)

    if verbose:
        print("Errore iniziale (trueError):", ci.trueError())

    for k in range(max_sweeps):
        ci.iterate(1)
        if verbose: 
            print(f"Iter {k+1}, trueError =", ci.trueError())
        if ci.isDone():
            if verbose:
                print("Convergenza raggiunta (isDone=True).")
            break

    tt = ci.tt

    # Ora la somma di f su tutti i path è semplicemente:
    price = tt.sum1()
    return price



In [9]:
import math
import itertools

def price_trinomial_call_bruteforce(
    N,
    S0,
    K,
    r,
    T,
    u, m, d,
    pu, pm, pd,
):
    disc = math.exp(-r * T)
    total = 0.0
    for path in itertools.product([0, 1, 2], repeat=N):
        S = S0
        prob = 1.0
        for move in path:
            if move == 0:
                S *= d
                prob *= pd
            elif move == 1:
                S *= m
                prob *= pm
            else:
                S *= u
                prob *= pu
        payoff = max(S - K, 0.0)
        total += disc * prob * payoff
    return total


In [ ]:
N = 5
S0 = 100.0
K  = 100.0
r  = 0.05
T  = 1.0

u  = 1.1
m  = 1.0
d  = 0.9
pu = 0.3
pm = 0.4
pd = 0.3

# exact = price_trinomial_call_bruteforce(N, S0, K, r, T, u, m, d, pu, pm, pd)
tt_price = price_trinomial_call_ttcross(
    N, S0, K, r, T, u, m, d, pu, pm, pd,
    bond_dim=50, reltol=1e-10, max_sweeps=30,
    verbose=True,
)

# print("Prezzo brute force =", exact)
print("Prezzo TT-cross    =", tt_price)


Errore iniziale (trueError): 0.15096349673614365
Iter 1, trueError = 0.04137070789262364
Iter 2, trueError = 0.03657286891320353
Convergenza raggiunta (isDone=True).
Prezzo brute force = 6.597107638465255
Prezzo TT-cross    = 6.6654768280158025


In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

# usi la tua funzione già definita
# from your_module import price_trinomial_call_ttcross


def benchmark_ttcross_pricing_error(
    S0, K, r, T,
    u, m, d, pu, pm, pd,
    P_ref,
    N_list=(25, 30, 50),
    D_min=30,
    D_max=200,
    n_D=10,
    n_runs=20,
    reltol=1e-8,
    max_sweeps=20,
    base_seed=1234,
):
    """
    Esegue il benchmark TTcross:
    - per N in N_list;
    - per D in valori equispaziati tra D_min e D_max;
    - per ogni coppia (N, D) esegue n_runs run indipendenti;
    - misura walltime e errore rispetto a P_ref[N].

    return:
        results[N]['D']           -> array dei D usati
        results[N]['time_mean']   -> walltime medio
        results[N]['err_median']  -> mediana degli errori
        results[N]['err_std']     -> std degli errori
    """

    D_values = np.linspace(D_min, D_max, n_D, dtype=int)

    # struttura dei risultati
    results = {
        N: {
            'D': D_values.copy(),
            'time_mean': [],
            'err_median': [],
            'err_std': [],
        }
        for N in N_list
    }

    for iN, N in enumerate(N_list):
        print(f"\n=== N = {N} ===")

        price_ref = P_ref[N]

        for jD, D in enumerate(D_values):
            print(f"  -> D = {D}  ({jD+1}/{len(D_values)})")

            run_times = []
            errors = []

            for run in range(n_runs):
                seed = base_seed + run + 100 * jD + 10_000 * iN

                t0 = time.perf_counter()
                price = price_trinomial_call_ttcross(
                    N,
                    S0=S0, K=K, r=r, T=T,
                    u=u, m=m, d=d,
                    pu=pu, pm=pm, pd=pd,
                    bond_dim=D,
                    reltol=reltol,
                    max_sweeps=max_sweeps,
                    verbose=False,
                )
                t1 = time.perf_counter()

                run_times.append(t1 - t0)
                errors.append(price - price_ref)

            run_times = np.array(run_times)
            errors = np.array(errors)

            results[N]['time_mean'].append(run_times.mean())
            results[N]['err_median'].append(np.median(errors))
            results[N]['err_std'].append(errors.std(ddof=1))

        # convertiamo in array
        for key in ['time_mean', 'err_median', 'err_std']:
            results[N][key] = np.array(results[N][key], float)

    return results


def plot_ttcross_pricing_error(results):
    """
    Plot stile Fig. 3 del paper.
    x-axis: walltime (log)
    y-axis: pricing error (mediana ± std)
    curve sopra e sotto.
    """

    fig, ax = plt.subplots(figsize=(8, 5))

    colors = {
        25: 'tab:blue',
        30: 'tab:orange',
        50: 'tab:green',
    }

    for N, data in results.items():
        t_mean  = data['time_mean']
        err_med = data['err_median']
        err_std = data['err_std']
        color   = colors.get(N, None)

        y_plus  = err_med + err_std
        y_minus = err_med - err_std

        ax.plot(
            t_mean, y_plus,
            label=f"N={N} median+std",
            color=color, linestyle='-',
            marker='o', linewidth=1.5
        )
        ax.plot(
            t_mean, y_minus,
            label=f"N={N} median-std",
            color=color, linestyle='--',
            marker='o', linewidth=1.5
        )

    ax.axhline(0.0, color='black', linewidth=1.0)
    ax.set_xscale('log')

    ax.set_xlabel("Walltime medio per run [s]")
    ax.set_ylabel("Pricing error (TTcross - reference)")
    ax.set_title("TTcross pricing error vs walltime")

    ax.legend()
    ax.grid(True, which='both', linestyle=':', alpha=0.7)
    plt.tight_layout()
    plt.show()


# Esempio di uso
if __name__ == "__main__":

    # parametri esempio (metti i tuoi veri parametri!)
    S0 = 100.0
    K  = 100.0
    r  = 0.05
    T  = 1.0
    u  = 1.1
    m  = 1.0
    d  = 0.9
    pu = 0.25
    pm = 0.50
    pd = 0.25

    # valori di riferimento (metti quelli veri!)
    P_ref = {
        25: 7.98985,
        30: 8.69942,
        50: 11.0844,
    }

    results = benchmark_ttcross_pricing_error(
        S0, K, r, T,
        u, m, d, pu, pm, pd,
        P_ref,
        N_list=(25, 30),
        D_min=20,
        D_max=50,
        n_D=10,
        n_runs=10,
        reltol=1e-8,
        max_sweeps=20,
    )

    plot_ttcross_pricing_error(results)
